In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('poster')

(Prochlorococcus marinus str. MIT 9313, complete genome
https://www.ncbi.nlm.nih.gov/nuccore/NC_005071.1/)

 

Prochlorococcus marinus MED4 complete genome
https://www.ncbi.nlm.nih.gov/nuccore/BX548174.1



![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
genome='MED4'

In [5]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO

In [6]:
# get all sequence records for the specified genbank file
recs = [rec for rec in SeqIO.parse(os.path.join('data', "MED4.gb"), "genbank")]

# print the number of sequence records that were extracted
len(recs)


1

In [7]:
int(recs[0].features[1].location.end)

1331

In [8]:
    merge_cols = ['contig_id', 'gene_id', 'location', 'type', 
                  'start', 'stop', 'strand',
                  'function', 'figfam',
                  'nucleotide_sequence', 'aa_sequence', 'genome', 'left', 'right']


In [9]:

def gb_to_record(i, genome='MED4'):
    q = i.qualifiers
    g = lambda x : ','.join(q.get(x, []))
    pmtid = [i for i in q.get('old_locus_tag',[]) if i.startswith('PMT')]
    record = {
        'contig_id' : genome, 
        'gene_id' : g('locus_tag'), 
        'pmt_id': ','.join(pmtid), 
        'type' : i.type, 
        'location' : g('locus_tag'),  
        'strand' : '+' if i.location.strand == 1 else '-',
        'start' : int(i.location.start),
        'stop' : int(i.location.end),
        'left' : int(i.location.start),
        'right' : int(i.location.end),
        'function' : g('product'), 
        'genome' : genome, 
        'old_locus_tag' : g('old_locus_tag'), 
        'product' : g('product'), 
        'db_xref' : g('db_xref'), 
        'protein_id' : g('protein_id'), 
        'figfam' : '',
        'nucleotide_sequence' : '', 
        'aa_sequence' : g('translation'), 
    }
    return record


In [10]:
types_to_collect = ['CDS', 'ncRNA', 'rRNA', 'regulatory', 'tRNA', 'tmRNA']
records = [gb_to_record(i) for i in recs[0].features if i.type in types_to_collect]

In [11]:
gdf = pd.DataFrame(records)

In [12]:

gdf.to_pickle(os.path.join('data', f'{genome}.ncbi.gb.pkl.gz'))


In [13]:
gdf.loc[gdf.type != 'CDS'].head(20)

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
63,MED4,RNA_37,,tRNA,RNA_37,-,65453,65535,65453,65535,tRNA-Leu,MED4,,tRNA-Leu,,,,,
66,MED4,PMED4_asRNA_00641,,ncRNA,PMED4_asRNA_00641,-,67628,67875,67628,67875,,MED4,,,,,,,
80,MED4,PMED4_asRNA_00771,,ncRNA,PMED4_asRNA_00771,-,79766,79930,79766,79930,,MED4,,,,,,,
135,MED4,RNA_1,,tRNA,RNA_1,+,122895,122967,122895,122967,tRNA-Asn,MED4,,tRNA-Asn,,,,,
239,MED4,PMED4_asRNA_02331,,ncRNA,PMED4_asRNA_02331,+,223721,223828,223721,223828,,MED4,,,,,,,
249,MED4,RNA_2,,tRNA,RNA_2,+,229472,229554,229472,229554,tRNA-Leu,MED4,,tRNA-Leu,,,,,
272,MED4,RNA_36,,tRNA,RNA_36,-,249344,249418,249344,249418,tRNA-Arg,MED4,,tRNA-Arg,,,,,
283,MED4,PMED4_asRNA_02731,,ncRNA,PMED4_asRNA_02731,+,258537,258669,258537,258669,,MED4,,,,,,,
290,MED4,RNA_35,,tRNA,RNA_35,-,264441,264517,264441,264517,tRNA-Met,MED4,,tRNA-Met,,,,,
327,MED4,RNA_34,,tRNA,RNA_34,-,296376,296452,296376,296452,tRNA-Phe,MED4,,tRNA-Phe,,,,,


In [14]:
genome = 'MED4'
accession = sf.get_accession(genome)
accession

'BX548174'

In [15]:
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')
df = sf.parse_orffinder_fasta(orf_fpath)

In [16]:
df['rast_left'] = df['left']
df['rast_right'] = df['right'] + 1

In [17]:
gdf.loc[gdf.location == 'AKG35_RS00020']

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence


In [18]:
df.loc[(df.rast_left > 4620) & (df.rast_left < 4630),  ].T

""
orfid
contig
left
right
start
stop
strand
aaseq
len_nn
len_aa


In [19]:
df = sf.add_gene_annotation_to_sorf_df(df, gdf)
df = sf.add_overlapping_genes(df, gdf)

In [20]:
summary_df = sf.gen_summary_df(df, genome)
df.to_pickle(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.pkl.gz'))
summary_df.to_csv(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.summary.csv.gz'))


In [21]:
summary_df

,index,otype,count,subset,genome
0,0,antisense,7381,all,MED4
1,1,in_frame,931,all,MED4
2,2,known,1495,all,MED4
3,3,out_frame,5715,all,MED4
4,4,overlap,749,all,MED4
5,5,standalone,798,all,MED4
6,0,antisense,7340,<100aa,MED4
7,1,in_frame,727,<100aa,MED4
8,2,known,395,<100aa,MED4
9,3,out_frame,5714,<100aa,MED4


In [22]:
orfid = 'ORF16879_BX548175.1:1658121:1657948'
orfid = 'ORF19962_BX548175.1:317208:317134'

In [23]:
df.loc[df.orfid.isin([orfid])].T

""
orfid
contig
left
right
start
stop
strand
aaseq
len_nn
len_aa


In [24]:
gdf.columns

Index(['contig_id', 'gene_id', 'pmt_id', 'type', 'location', 'strand', 'start',
       'stop', 'left', 'right', 'function', 'genome', 'old_locus_tag',
       'product', 'db_xref', 'protein_id', 'figfam', 'nucleotide_sequence',
       'aa_sequence'],
      dtype='object')

In [25]:
gdf.loc[(gdf.left > 317100) & (gdf.right < 317220)]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence


In [26]:
gdf.loc[gdf['product'].str.contains('hypothetical protein')]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
1,MED4,PMM0002,,CDS,PMM0002,+,1332,2040,1332,2040,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0002,EnsemblGenomes-Tr:CA...",CAE18461.1,,,MKLPKEFLLSELLRHNVKGNATLNYGKGENVWMHPPVHRILGWYSR...
5,MED4,PMM0006,,CDS,PMM0006,-,8415,9270,8415,9270,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0006,EnsemblGenomes-Tr:CA...",CAE18465.1,,,MKKSLLKILFFSIISSHVFIAESLKALIPYYYLPETKSLQKQGLSI...
6,MED4,PMM0007,,CDS,PMM0007,-,9274,10219,9274,10219,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0007,EnsemblGenomes-Tr:CA...",CAE18466.1,,,MRNMIQNKKEFSEKLKKRAIFEGFAVSGIASIPGSSRVKLRTQALE...
7,MED4,PMM0008,,CDS,PMM0008,+,10366,11104,10366,11104,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0008,EnsemblGenomes-Tr:CA...",CAE18467.1,,,MVESTQSQDSNLGTRLQQDLKNDLIAGLLVVIPLATTIWLSSIVSK...
13,MED4,PMM0014,,CDS,PMM0014,-,16679,17684,16679,17684,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0014,EnsemblGenomes-Tr:CA...",CAE18473.1,,,MKSYQINTINNLHKISVAPMMDCTDKHFRMIIRKISSKALLYTEMI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,MED4,PMM1681,,CDS,PMM1681,-,1613534,1614146,1613534,1614146,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM1681,EnsemblGenomes-Tr:CA...",CAE20140.1,,,MTLIFENIYNLAYFFNTNIGIFAFILLYILIVLLILPASWLSLLSG...
2005,MED4,PMM1698,,CDS,PMM1698,+,1633926,1634376,1633926,1634376,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM1698,EnsemblGenomes-Tr:CA...",CAE20157.1,,,MLTFINYQSKLFPTIEDVDRKSYGTLFYCLSLFILIYLYWEKDPTS...
2015,MED4,PMM2020,,CDS,PMM2020,-,1646198,1646384,1646198,1646384,Conserved hypothetical protein,MED4,,Conserved hypothetical protein,"EnsemblGenomes-Gn:PMM2020,EnsemblGenomes-Tr:CA...",CAP16525.1,,,MFDKENLKYFLIWPMSVLLAIFFKYYGFLKPDFLLINNYLVLLLVC...
2016,MED4,PMM1708,,CDS,PMM1708,+,1646438,1646717,1646438,1646717,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM1708,EnsemblGenomes-Tr:CA...",CAE20167.1,,,MDSNKLKIRLNEIAEVNPALTCYHREDPAPVLPLREEPDLLSWLED...


In [27]:
gdf.loc[gdf.protein_id.str.contains('WP_011129965')]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence


In [28]:
df.loc[df.orfid== 'ORF30023_BX548175.1:644083:643928']

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,overlap_strand,overlap_gene_type,overlap_count,is_same_strand,is_out_of_frame,is_inside,is_upstream,is_downstream,overlap_type,otype


In [29]:
summary_df

,index,otype,count,subset,genome
0,0,antisense,7381,all,MED4
1,1,in_frame,931,all,MED4
2,2,known,1495,all,MED4
3,3,out_frame,5715,all,MED4
4,4,overlap,749,all,MED4
5,5,standalone,798,all,MED4
6,0,antisense,7340,<100aa,MED4
7,1,in_frame,727,<100aa,MED4
8,2,known,395,<100aa,MED4
9,3,out_frame,5714,<100aa,MED4


In [30]:
gdf.left.value_counts()

313060     2
228280     2
178177     1
1397443    1
1108672    1
          ..
750968     1
1412467    1
79218      1
810353     1
202751     1
Name: left, Length: 2023, dtype: int64

In [31]:
gdf.head()

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
0,MED4,PMM0001,,CDS,PMM0001,+,173,1331,173,1331,"DNA polymerase III, beta chain",MED4,,"DNA polymerase III, beta chain","EnsemblGenomes-Gn:PMM0001,EnsemblGenomes-Tr:CA...",CAE18460.1,,,MEIVCNQNEFNYAIQLVSKAVASRPTHPILANLLLTADQGTNKISL...
1,MED4,PMM0002,,CDS,PMM0002,+,1332,2040,1332,2040,conserved hypothetical protein,MED4,,conserved hypothetical protein,"EnsemblGenomes-Gn:PMM0002,EnsemblGenomes-Tr:CA...",CAE18461.1,,,MKLPKEFLLSELLRHNVKGNATLNYGKGENVWMHPPVHRILGWYSR...
2,MED4,PMM0003,,CDS,PMM0003,+,2043,4383,2043,4383,phosphoribosylformylglycinamidine synthetase II,MED4,,phosphoribosylformylglycinamidine synthetase II,"EnsemblGenomes-Gn:PMM0003,EnsemblGenomes-Tr:CA...",CAE18462.1,,,MIDSSSNNTYDVNESLKVENLTRDDYEEICKRLGRKPNRTELGMFG...
3,MED4,PMM0004,,CDS,PMM0004,+,4429,5890,4429,5890,Glutamine amidotransferase class-II:Phosphorib...,MED4,,Glutamine amidotransferase class-II:Phosphorib...,"EnsemblGenomes-Gn:PMM0004,EnsemblGenomes-Tr:CA...",CAE18463.1,,,MCGIVGIVSCDDVNQQIYDSLLLLQHRGQDSTGIATMENTVFHIHK...
4,MED4,PMM0005,,CDS,PMM0005,-,5886,8328,5886,8328,"DNA gyrase/topoisomerase IV, subunit A",MED4,,"DNA gyrase/topoisomerase IV, subunit A","EnsemblGenomes-Gn:PMM0005,EnsemblGenomes-Tr:CA...",CAE18464.1,,,MTKEKFTSISLQEEMQRSYLEYAMSVIIGRALPDARDGLKPVQRRI...


In [32]:
gdf.type.value_counts()

CDS      1960
tRNA       37
ncRNA      24
rRNA        4
Name: type, dtype: int64